In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
import pandas as pd
import re
import string
from textblob import TextBlob, Word, Blobber
import nltk
from nltk.corpus import stopwords
# print(stopwords.words('english'))
import plotly.express as px
from nltk.stem import WordNetLemmatizer 
from tensorflow.keras.preprocessing.text import one_hot
nltk.download('stopwords')
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from sklearn.model_selection import GridSearchCV, cross_val_score, RepeatedStratifiedKFold, RandomizedSearchCV, LeavePOut, StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/NLP/NLP project/train.csv')

In [ ]:
df = df.dropna()
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
from tqdm.notebook import tqdm, trange
from nltk.stem import PorterStemmer
!pip install autocorrect
from autocorrect import Speller
from autocorrect import Speller

spell = Speller(lang='en')

porter = PorterStemmer()
from nltk.tokenize import sent_tokenize, word_tokenize
def lower_text(text):
    return text.lower()

def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

def remove_numbers(text):
    text_cleaned = re.sub('[^a-zA-Z]',' ',text)
    return text_cleaned

def remove_stopwords(text):
    stoplist = stopwords.words('english')
    sps = stoplist
    return " ".join([word for word in str(text).split() if word not in sps])

def autospell(text):
    return " ".join([spell(word) for word in text.split()])

corpus=[]
def get_corpus(sentence):
    cps = []
    token_words=word_tokenize(sentence)
    for word in token_words:
        corpus.append(word)
    return sentence



In [ ]:
df['text'] = df['text'].apply(remove_stopwords)
df['title'] = df['title'].apply(remove_stopwords)
df['author'] = df['author'].apply(remove_stopwords)

df['text'] = df['text'].apply(remove_numbers)
df['title'] = df['title'].apply(remove_numbers)
df['author'] = df['author'].apply(remove_numbers)

df['text'] = df['text'].apply(lemmatization)
df['title'] = df['title'].apply(lemmatization)
df['author'] = df['author'].apply(lemmatization)

df['text'] = df['text'].apply(lower_text)
df['title'] = df['title'].apply(lower_text)
df['author'] = df['author'].apply(lower_text)

# df['text'] = df['text'].apply(get_corpus)
# df['title'] = df['title'].apply(get_corpus)
# df['author'] = df['author'].apply(get_corpus)
# corpus = set(corpus)

In [ ]:
# dropping length 0, 1, 2 texts
ohr = []
for index in range(len(df)):
    # ohr.append(len(onehot_rep_test[index]))
    if len(df['text'][index])==0 or len(df['text'][index])==1 or len(df['text'][index])==2:
        ohr.append(index)
print(len(ohr), len(df))
df = df.drop(ohr)        
df = df.sample(frac=1).reset_index(drop=True)
print(len(df))

93 18285
18192


In [ ]:
df['titleplustext'] = df['text'] + " " +df['title']
df['authorplustext'] = df['text'] + " " +df['author']  
df['titleauthortext'] = df['text'] + " " +df['title'] + " " +df['author']

In [ ]:
df.to_csv("/content/drive/MyDrive/NLP/NLP project/filtered_df.csv", index=False)

In [ ]:
df

,id,title,author,text,label,titleplustext,authorplustext,titleauthortext
0,16753,rick perry ex governor texas is trump s pick e...,coral davenport,washington donald j trump plan name rick perry...,0,washington donald j trump plan name rick perry...,washington donald j trump plan name rick perry...,washington donald j trump plan name rick perry...
1,6909,deepwater horizon continues impact public health,admin,deepwater horizon continues impact public heal...,1,deepwater horizon continues impact public heal...,deepwater horizon continues impact public heal...,deepwater horizon continues impact public heal...
2,10842,nikki haley puts u n notice u s is taking name...,somini sengupta,united nations the american ambassador united ...,0,united nations the american ambassador united ...,united nations the american ambassador united ...,united nations the american ambassador united ...
3,19326,what hillary clinton india not much here s why,saurav dutt,bill clinton cabinet member pursued anti india...,1,bill clinton cabinet member pursued anti india...,bill clinton cabinet member pursued anti india...,bill clinton cabinet member pursued anti india...
4,17547,donald trump announces urban revitalization pl...,iwb,dr darrell scott pastordscott october seriousl...,1,dr darrell scott pastordscott october seriousl...,dr darrell scott pastordscott october seriousl...,dr darrell scott pastordscott october seriousl...
...,...,...,...,...,...,...,...,...
18187,16492,gary johnson s what is aleppo flub amplifies s...,alan rappeport,when accepted libertarian party s nomination m...,0,when accepted libertarian party s nomination m...,when accepted libertarian party s nomination m...,when accepted libertarian party s nomination m...
18188,1327,mike allen politico s newsletter pioneer is ha...,jim rutenberg,siren washington to lose its daily bard at a m...,0,siren washington to lose its daily bard at a m...,siren washington to lose its daily bard at a m...,siren washington to lose its daily bard at a m...
18189,1700,student sues police over baton encounter brook...,rick rojas,an brooklyn high school student sued new york ...,0,an brooklyn high school student sued new york ...,an brooklyn high school student sued new york ...,an brooklyn high school student sued new york ...
18190,7557,hillary clinton united nations same open borde...,kaitlyn stegall,october hillary clinton united nations same op...,1,october hillary clinton united nations same op...,october hillary clinton united nations same op...,october hillary clinton united nations same op...


In [ ]:
df  = pd.read_csv("/content/drive/MyDrive/NLP/NLP project/filtered_df.csv")

In [ ]:
X = df['text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
X_train = X_train.sample(frac=1).reset_index(drop=True)
X_test = X_test.sample(frac=1).reset_index(drop=True)

In [ ]:
def return_corpus(all_sentences):
    corpus=[]
    for sentence in all_sentences:
        token_words=word_tokenize(sentence)
        corpus.extend(token_words)
    return corpus

In [ ]:
train_corpus = return_corpus(X_train)
test_corpus = return_corpus(X_test)
train_corpus = set(train_corpus)
test_corpus = set(test_corpus)
len(train_corpus), len(test_corpus)

(121892, 60713)

In [ ]:
import gensim
import gensim.downloader as api
from gensim.models import Word2Vec

def get_embs_vocab(model, corpus):
    embeddings = {}
    for word in corpus:
        word = word.lower()
        try:
            embeddings[word] = model[word]
        except:
            words = list(gensim.utils.tokenize(word))
            try:
                embs = [model[i] for i in words]
                emb = sum(embs)
                if emb ==0:
                    embeddings[word] = np.zeros(sh)
                else:
                    embeddings[word] = emb
            except:
                embeddings[word] = np.zeros(vec_king.shape)
    return embeddings
wv = api.load('word2vec-google-news-300')
# get_embs_vocab(wv, corpus)

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
vec_king = wv['king']
sh = vec_king.shape

In [ ]:
train_word2vec_embeddings = get_embs_vocab(wv, train_corpus)
test_word2vec_embeddings = get_embs_vocab(wv, test_corpus)
len(train_word2vec_embeddings), len(test_word2vec_embeddings)

(121892, 60713)

In [ ]:
paddinglen = 200

In [ ]:
train_embed = []
for sentence in X_train:
    token_words=word_tokenize(sentence)
    if len(token_words)<paddinglen:
        tobeadded = [np.zeros(sh)]*(paddinglen-len(token_words))
        # print(len(tobeadded), tobeadded[0].shape)
        sentence_embed = [train_word2vec_embeddings[word] for word in token_words]
        sentence_embed.extend(tobeadded)
    else:
        token_words=token_words[:paddinglen]
        sentence_embed = [train_word2vec_embeddings[word] for word in token_words]
    train_embed.append(sentence_embed)
train_embed = np.asarray(train_embed)

In [ ]:
test_embed = []
for sentence in X_test:
    token_words=word_tokenize(sentence)
    if len(token_words)<paddinglen:
        tobeadded = [np.zeros(sh)]*(paddinglen-len(token_words))
        # print(len(tobeadded), tobeadded[0].shape)
        sentence_embed = [test_word2vec_embeddings[word] for word in token_words]
        sentence_embed.extend(tobeadded)
    else:
        token_words=token_words[:paddinglen]
        sentence_embed = [test_word2vec_embeddings[word] for word in token_words]
    test_embed.append(sentence_embed)
test_embed = np.asarray(test_embed)

In [ ]:
train_embed.shape, test_embed.shape

In [ ]:
X_train = train_embed.reshape(train_embed.shape[0], train_embed.shape[1]*train_embed.shape[2])
X_test = test_embed.reshape(test_embed.shape[0], test_embed.shape[1]*test_embed.shape[2])
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
np.save(X_train, "/content/drive/MyDrive/NLP/NLP project/train_x.npy")
np.save(X_test, "/content/drive/MyDrive/NLP/NLP project/test_x.npy")
np.save(y_train, "/content/drive/MyDrive/NLP/NLP project/train_y.npy")
np.save(y_test, "/content/drive/MyDrive/NLP/NLP project/test_y.npy")

In [ ]:
from sklearn.pipeline import make_pipeline
clf = make_pipeline(SVC(gamma='auto'))
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')


In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

In [ ]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

accuracy:   0.552
              precision    recall  f1-score   support

           0       0.57      0.89      0.69      2076
           1       0.43      0.11      0.18      1581

    accuracy                           0.55      3657
   macro avg       0.50      0.50      0.43      3657
weighted avg       0.51      0.55      0.47      3657

confusion matrix:
[[1842  234]
 [1404  177]]


In [ ]:
clf = AdaBoostClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

accuracy:   0.546
              precision    recall  f1-score   support

           0       0.57      0.82      0.67      2076
           1       0.44      0.18      0.26      1581

    accuracy                           0.55      3657
   macro avg       0.51      0.50      0.47      3657
weighted avg       0.51      0.55      0.49      3657

confusion matrix:
[[1707  369]
 [1290  291]]


In [ ]:
clf = MLPClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

accuracy:   0.467
              precision    recall  f1-score   support

           0       0.56      0.28      0.37      2076
           1       0.43      0.72      0.54      1581

    accuracy                           0.47      3657
   macro avg       0.50      0.50      0.45      3657
weighted avg       0.51      0.47      0.44      3657

confusion matrix:
[[ 573 1503]
 [ 446 1135]]


In [ ]:
clf = GradientBoostingClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

accuracy:   0.562
              precision    recall  f1-score   support

           0       0.57      0.93      0.71      2076
           1       0.46      0.08      0.13      1581

    accuracy                           0.56      3657
   macro avg       0.51      0.50      0.42      3657
weighted avg       0.52      0.56      0.46      3657

confusion matrix:
[[1936  140]
 [1462  119]]


In [ ]:
clf = PassiveAggressiveClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

accuracy:   0.456
              precision    recall  f1-score   support

           0       0.55      0.23      0.32      2076
           1       0.43      0.75      0.55      1581

    accuracy                           0.46      3657
   macro avg       0.49      0.49      0.43      3657
weighted avg       0.50      0.46      0.42      3657

confusion matrix:
[[ 478 1598]
 [ 390 1191]]


LSTM on tokenized text


In [ ]:
max_features=4500

In [ ]:
from nltk.tokenize import sent_tokenize,word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
X = df['text']

In [ ]:
tokenizer = Tokenizer(num_words = max_features, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower = True, split = ' ')
tokenizer.fit_on_texts(texts = X)
X = tokenizer.texts_to_sequences(texts = X)

In [ ]:
X = pad_sequences(sequences = X, maxlen = max_features, padding = 'pre')

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
lstm_model = Sequential(name = 'lstm_nn_model')
lstm_model.add(layer = Embedding(input_dim = max_features, output_dim = 120, name = '1st_layer'))
lstm_model.add(layer = LSTM(units = 120, dropout = 0.2, recurrent_dropout = 0.2, name = '2nd_layer'))
lstm_model.add(layer = Dropout(rate = 0.5, name = '3rd_layer'))
lstm_model.add(layer = Dense(units = 120,  activation = 'relu', name = '4th_layer'))
lstm_model.add(layer = Dropout(rate = 0.5, name = '5th_layer'))
lstm_model.add(layer = Dense(units = len(set(y)),  activation = 'sigmoid', name = 'output_layer'))
# compiling the model
lstm_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
lstm_model_fit = lstm_model.fit(X_train1, y_train1, epochs = 1)

 83/383 [=====>........................] - ETA: 1:12:41 - loss: 0.5508 - accuracy: 0.6962

In [ ]:
y_pred = lstm_model_fit.predict(X_test1)
score1 = metrics.accuracy_score(y_test1, y_pred)
print("accuracy:   %0.3f" % score1)
print(metrics.classification_report(y_test1, y_pred))
print("confusion matrix:")
print(metrics.confusion_matrix(y_test1, y_pred))

GRU

In [ ]:
gru_model = Sequential(name = 'gru_nn_model')
gru_model.add(layer = Embedding(input_dim = max_features, output_dim = 120, name = '1st_layer'))
gru_model.add(layer = GRU(units = 120, dropout = 0.2, 
                          recurrent_dropout = 0.2, recurrent_activation = 'relu', 
                          activation = 'relu', name = '2nd_layer'))
gru_model.add(layer = Dropout(rate = 0.4, name = '3rd_layer'))
gru_model.add(layer = Dense(units = 120, activation = 'relu', name = '4th_layer'))
gru_model.add(layer = Dropout(rate = 0.2, name = '5th_layer'))
gru_model.add(layer = Dense(units = len(set(y)), activation = 'softmax', name = 'output_layer'))
# compiling the model
gru_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
gru = gru_model.fit(X_train, y_train, epochs = 1)

In [ ]:
y_pred = gru.predict(X_test)
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)
print(metrics.classification_report(y_test, y_pred))
print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))